# Power Plant Data Product: Data Loading Step 2

This notebook covers the second step of the pipeline:
* Load versioned source data from Pachyderm
* Get data into a dataframe 
* Create a Trino client and load the data

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
# Report exceptions only in cells output
%xmode Minimal

In [ ]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import socket
from pyspark import SparkConf , SparkContext, SparkFiles
from pyspark.sql import SparkSession, Row
import pandas as pd
import trino
from sqlalchemy import create_engine, text

Define Environment and Execution Variables

In [ ]:
# Load environment variables from credentials.env
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Read the data directory holding raw source data pulled from DVC, and list all files

In [ ]:
path = '../data/global_power_plant_database_v_1_3/'
files = os.listdir(path)
for f in files:
    print(f)

In [ ]:
srcdata = str(pathlib.Path('../data/global_power_plant_database_v_1_3/global_power_plant_database.csv').absolute())
print(srcdata)

Create a connection to Spark cluster

In [ ]:
SPARK_SERVER = os.environ['SPARK_SERVER']
SPARK_CONNECT_PORT = os.environ['SPARK_CONNECT_PORT']

In [ ]:
SPARK_CONNECT_URL = f'sc://{SPARK_SERVER}:{SPARK_CONNECT_PORT}'
conf = SparkConf() \
    .setAppName("Spark App") \
    .setMaster(SPARK_CONNECT_URL) \
    .set("spark.driver.host", socket.gethostbyname(socket.gethostname()))\
    .set("spark.driver.port", "10027") 

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [ ]:
spark

Create a Spark dataframe from the CSV file (temporary workaround via pandas)

In [ ]:
df = pd.read_csv(srcdata, low_memory=False).convert_dtypes()

In [ ]:
print(df.info(verbose=True))

In [ ]:
spark_df = spark.createDataFrame(df)

In [ ]:
df.head()

In [ ]:
spark_df.printSchema()

Write the dataframe into Iceberg

In [ ]:
try:
    spark_df.writeTo("datamesh.wri_gppd.gppd_source").createOrReplace()
    print("Data written to Iceberg table successfully." )
except Exception as e:
    print("Error writing data to Iceberg table:", str(e))

Connect to Trino and ensure the data is loaded

In [ ]:
# Define execution variables for Iceberg / Trino
ingest_catalog = 'hive'
ingest_schema = 'wri_gppd'
ingest_table = 'gppd_source'

In [ ]:
# Create a connection to Trino using an SQLAlchemy Engine
# Documentation: https://docs.sqlalchemy.org/en/20/core/engines.html#sqlalchemy.create_engine
engine = create_engine(
    'trino://' + os.environ['TRINO_USER'] + ':' + os.environ['TRINO_PASSWD']
    + '@' + os.environ['TRINO_HOST'] + ':' + os.environ['TRINO_PORT']
)

In [ ]:
engine.connect

In [ ]:
# Show existing catalogs in Trino to make sure the connection works fine
with engine.connect() as connection:
    result = connection.execute(text('show catalogs'))
    for row in result:
        print(row)

In [ ]:
# Show available schemas
schema_show_sql = f"""
show schemas in {ingest_catalog}
"""
with engine.connect() as connection:
    result = connection.execute(text(schema_show_sql))
    for row in result:
        print(row)

In [ ]:
# Show available tables
tables_show_sql = f"""
show tables from {ingest_catalog}.{ingest_schema}
"""
with engine.connect() as connection:
    result = connection.execute(text(tables_show_sql))
    for row in result:
        print(row)

In [ ]:
# Query table to verify insertion was successful
select_query = f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(select_query)
with engine.connect() as connection:
    result = connection.execute(text(select_query)).fetchmany(10)
    for row in result:
        print(row)